In [24]:
import pandas as pd
import json
import re
import itertools
import MeCab

In [70]:
test_df = pd.read_pickle("../../data/test_IOB_repl_compound_fix_cat_headlines.pkl")

In [208]:
predict_material_df = pd.read_pickle("../../dump/pred_raw-material_using_compound-list_with_tag_seq.pkl")
predict_production_df = pd.read_pickle("../../dump/pred_production_using_compound-list_with_tag_seq.pkl")

In [201]:
with open("../../output/result/production_using_compound-list.json", 'r') as f:
    extracted_production_data = json.load(f)

with open("../../output/result/raw-material_using_compound-list.json", 'r') as f:
    extracted_material_data = json.load(f)

In [321]:
m_parser = MeCab.Tagger("-u /Users/dontsentouin/mecab-user-dic/compound.dic -Owakati")

flatten = lambda l: [i for sl in l for i in sl]

def exact_match(true, predict):
    return list(set(predict) & set(true))

def exact_false(true, predict):
    partial_match_set = set([p for t, p in partial_match(true, predict)])
    
    return list(set(predict) - set(true) - partial_match_set)

def failure(true, predict):
    partial_match_set = set([t for t, p in partial_match(true, predict)])
    
    return list(set(true) - set(predict) - partial_match_set)

def partial_match(true, pred, return_format=0):
    # 両方返す
    if return_format is 0:
        return [(t, p) for t, p in itertools.product(true, pred) \
                if (re.search(fr'{re.escape(t)}', p) or re.search(fr'{re.escape(p)}', t)) and t != p]
    # Trueのみ
    elif return_format is 1:
        return [t for t, p in itertools.product(true, pred) \
                if (re.search(fr'{re.escape(t)}', p) or re.search(fr'{re.escape(p)}', t)) and t != p]
    # 抽出データのみ
    else:
        return [p for t, p in itertools.product(true, pred) \
                if (re.search(fr'{re.escape(t)}', p) or re.search(fr'{re.escape(p)}', t)) and t != p]

def parse(s):
    return m_parser.parse(s).strip().split()

In [186]:
def is_contain_sent(sentence, annotation):
    if annotation:
        return re.search(annotation, sentence)
    return False

def join_words(words, escape=False):
    if escape:
        return ''.join([re.escape(w) for w in words])
    else:
        return ''.join(words)

def get_contained_sent(annotations, doc_df): 
    annotations = sorted(annotations, key=len, reverse=True)
    
    s_list = []
    for annotation in annotations:
        parsed_annotation = parse(annotation)
        for _, row in doc_df.iterrows():
            if is_contain_sent(join_words(row['words'], escape=False), join_words(parsed_annotation, escape=True)):
                s_list.append((annotation, row['sentence']))
                
    return s_list

## 抽出できなかったデータ

In [179]:
failures_dict = {}
for key, data in extracted_production_data.items():
    predict = data['predict']
    true = data['true']
    
    failures_sent = get_contained_sent(failure(true, predict), test_df.loc[test_df._id == key])
    match_list = list(set([t for t, p in partial_match(true, predict)] + true))
    
    failures_dict[key] = {
        'title': data['title']
        , 'failure': failures_sent
        #, 'failure': list(set([(t, s) for t, s in failures_sent for m in match_list if m in s]))
    }

In [184]:
flatten(
    [list(set([f_data[0] for f_data in v['failure']])) 
     for v in failures_dict.values() if v['failure']
    ]
)

['化石燃料由来のエチレンの水和反応等の有機合成手法によっても製造される。',
 'ソハイオ法 によるアクリロニトリル製造の際の副産物として得られる',
 'シアン化ナトリウムに酸を加える方法',
 'アンドルソフ法',
 '対称エーテルは、酸触媒の存在下にアルコールの分子間脱水縮合で得られる。',
 '脳内でグルタミン酸のα位のカルボキシル基がグルタミン酸脱炭酸酵素との反応により除かれることによって生成される',
 'トルエンから合成',
 '2-クロロトルエンからも作ることもできる',
 '生体内では、尿素回路によりアンモニアから尿素が産生される',
 'ヴェーラー合成',
 'ケイ皮酸脱炭酸酵素によってケイ皮酸から合成',
 'グラブス触媒を用いたオレフィンメタセシスによる閉環反応を鍵反応とし、13個の連結したエーテル環構造を効率的に合成する',
 'ポリケチド経路によって生合成',
 'ある種の藻類（有毒渦鞭毛藻）がつくり魚類に蓄積される',
 '硫酸と水酸化カルシウムが中和して生成',
 'γ-ブチロラクトンとメチルアミンとを縮合させて得る',
 'アミドの酸素を硫黄に置換して合成する',
 'アミドと五硫化二リン (P2S5) を加熱する方法が取られていたが、近年ではより穏和な条件で反応が進行するローソン試薬を用いるケースが増えている',
 '生体内ではグルコース-1-デヒドロゲナーゼの作用によりグルコースから変換される。',
 '前駆体から小分子を脱離させる手法',
 '反応触媒としては、塩化亜鉛の他にスルホン酸も用いられる',
 '塩化ロジウム三水和物を塩化水素中で360℃で加熱分解する',
 '塩化水素中の180℃の加熱',
 'プソイドイオノン から合成する',
 '塩化白金酸(H2PtCl6)または塩化白金酸アンモニウム((NH4)2PtCl6)を硝酸ナトリウムに溶融',
 'その生合成にはいくつかの潜在的ルートがあるが、もっともよく特徴付けられているのは、オートタキシンと呼ばれるリゾホスホリパーゼDによるものであり、リゾホスファチジルコリンからコリンを除去する。',
 'スクロースを原料として乳酸菌が生産',
 'Leuconostoc mesenteroidesが生産する高分子デキストランを部分的に加水分解し、これを精製する',
 '90℃

In [181]:
with open("../../output/error_analysis/production_failures_with_sentence.json", 'w') as f:
    json.dump(failures_dict, f, ensure_ascii=False)

## 製造方法&原材料

In [295]:
predict_production_df.groupby('_id', as_index=False).extracted.sum().apply(
    lambda x: len(exact_match(extracted_production_data[x._id]['true'], x.extracted))
    , axis=1
).sum()

11

### 製造方法の抽出結果のカウント

#### 原材料抽出無し

In [334]:
none_material_exist_prod_df = \
pd.merge(
    predict_material_df, predict_production_df, on=['_id', 'sentence'], how='right'
).drop(columns='words_y').rename(
    columns={
        'words_x': 'words'
        , 'pred_seq_x': 'pred_material_seq'
        , 'extracted_x': 'extracted_material'
        , 'pred_seq_y': 'pred_production_seq'
        , 'extracted_y': 'extracted_prodction'
    }
)
none_material_exist_prod_df = none_material_exist_prod_df[none_material_exist_prod_df.extracted_material.isna()]
print(len(none_material_exist_prod_df))

23


In [226]:
# 抽出件数の確認
none_material_exist_prod_df.groupby('_id', as_index=False).extracted_prodction.sum().apply(
    lambda x: len(set(x.extracted_prodction))
    , axis=1
).sum()

23

In [331]:
none_material_exist_prod_df.groupby('_id', as_index=False).extracted_prodction.sum().apply(
    lambda x: len(set(#partial_match(extracted_production_data[x._id]['true'], set(x.extracted_prodction), return_format=2)
                  exact_match(extracted_production_data[x._id]['true'], set(x.extracted_prodction))
                 ))
    , axis=1
).sum()

1

#### 原材料抽出有り

In [335]:
exist_material_exist_prod_df = \
pd.merge(
    predict_material_df, predict_production_df, on=['_id', 'sentence']
).drop(columns='words_y').rename(
    columns={
        'words_x': 'words'
        , 'pred_seq_x': 'pred_material_seq'
        , 'extracted_x': 'extracted_material'
        , 'pred_seq_y': 'pred_production_seq'
        , 'extracted_y': 'extracted_prodction'
    }
)
print(len(exist_material_exist_prod_df))

64


In [279]:
# 抽出件数の確認
exist_material_exist_prod_df.groupby('_id', as_index=False).extracted_prodction.sum().apply(
    lambda x: len(set(x.extracted_prodction))
    , axis=1
).sum()

70

In [333]:
exist_material_exist_prod_df.groupby('_id', as_index=False).extracted_prodction.sum().apply(
    lambda x: len(set(#partial_match(extracted_production_data[x._id]['true'], set(x.extracted_prodction), return_format=2)
                  exact_match(extracted_production_data[x._id]['true'], set(x.extracted_prodction))
                 ))
    , axis=1
).sum()

10

### 原材料の抽出結果のカウント

#### 製造方法抽出無し

In [337]:
exist_material_none_prod_df = \
pd.merge(
    predict_material_df, predict_production_df, on=['_id', 'sentence'], how='left'
).drop(columns='words_y').rename(
    columns={
        'words_x': 'words'
        , 'pred_seq_x': 'pred_material_seq'
        , 'extracted_x': 'extracted_material'
        , 'pred_seq_y': 'pred_production_seq'
        , 'extracted_y': 'extracted_prodction'
    }
)
exist_material_none_prod_df = exist_material_none_prod_df[exist_material_none_prod_df.extracted_prodction.isna()]
print(len(exist_material_none_prod_df))

180


In [340]:
# 抽出件数の確認
exist_material_none_prod_df.groupby('_id', as_index=False).extracted_material.sum().apply(
    lambda x: len(set(x.extracted_material))
    , axis=1
).sum()

193

In [342]:
exist_material_none_prod_df.groupby('_id', as_index=False).extracted_material.sum().apply(
    lambda x: len(set(partial_match(extracted_material_data[x._id]['true'], set(x.extracted_material), return_format=2)
                  #exact_match(extracted_material_data[x._id]['true'], set(x.extracted_material))
                 ))
    , axis=1
).sum()

20

#### 製造方法抽出有り

In [344]:
# 抽出件数の確認
exist_material_exist_prod_df.groupby('_id', as_index=False).extracted_material.sum().apply(
    lambda x: len(set(x.extracted_material))
    , axis=1
).sum()

115

In [346]:
exist_material_exist_prod_df.groupby('_id', as_index=False).extracted_material.sum().apply(
    lambda x: len(set(#partial_match(extracted_material_data[x._id]['true'], set(x.extracted_material), return_format=2)
                  exact_match(extracted_material_data[x._id]['true'], set(x.extracted_material))
                 ))
    , axis=1
).sum()

63